In [3]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd

whole = pd.read_csv('drive/Shared drives/IST 597/Data-FE/whole_df_Twitter1_final.csv')
#whole = whole.drop(columns=['Unnamed: 0', 'Unnamed: 0.1','Unnamed: 0.1.1','num_comments','selftext','created_utc','score','title','day_feature','time_feature','month_feature','clean_text','score'])
whole = whole.drop(columns=['Unnamed: 0', '0','1','2','3','4','5','6','7','8','9','score','month_feature','year_feature','time_feature','day_feature','clean_text'])

In [7]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
import json
import numpy as np
from sklearn.ensemble import AdaBoostRegressor
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from sklearn.preprocessing import MinMaxScaler



/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [14]:

model_performance_dict = dict()
#whole = pd.read_csv('drive/Shared drives/IST 597/Data-FE/submission_2015_2017.tsv',sep='\t')
whole1 = whole


X_train = whole[whole['type']=='train']
X_val = whole[whole['type']=='val']
X_train = X_train.append(X_val)
X_test = whole[whole['type']=='test']

y_train = X_train['z score']
X_train = X_train.drop(['type','z score'], axis=1)
y_test = X_test['z score']
X_test = X_test.drop(['type','z score'], axis=1)
"""

X = whole.drop(['z score'], axis=1)
y = whole['z score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
#X_train
"""

#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

"\n\nX = whole.drop(['z score'], axis=1)\ny = whole['z score']\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)\n#X_train\n"

In [0]:


def model_diagnostics(model, pr=True):
    """
    Returns and prints the R-squared, RMSE and the MAE for a trained model
    """
    y_predicted = model.predict(X_test)
    r2 = r2_score(y_test, y_predicted)
    mse = mean_squared_error(y_test, y_predicted)
    mae = mean_absolute_error(y_test, y_predicted)
    if pr:
        print(f"R-Sq: {r2:.4}")
        print(f"RMSE: {np.sqrt(mse)}")
        print(f"MAE: {mae}")
    
    return [r2,np.sqrt(mse),mae]
def plot_residuals(y_test, y_predicted):
    """"
    Plots the distribution for actual and predicted values of the target variable. Also plots the distribution for the residuals
    """
    fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, sharey=True)
    sns.distplot(y_test, ax=ax0, kde = False)
    ax0.set(xlabel='Test scores')
    sns.distplot(y_predicted, ax=ax1, kde = False)
    ax1.set(xlabel="Predicted scores")
    plt.show()
    fig, ax2 = plt.subplots()
    sns.distplot((y_test-y_predicted), ax = ax2,kde = False)
    ax2.set(xlabel="Residuals")
    plt.show()
def y_test_vs_y_predicted(y_test,y_predicted):
    """
    Produces a scatter plot for the actual and predicted values of the target variable
    """
    fig, ax = plt.subplots()
    ax.scatter(y_test, y_predicted)
    ax.set_xlabel("Test Scores")
    ax.set_ylim([-75, 1400])
    ax.set_ylabel("Predicted Scores")
    plt.show()
def get_feature_importance(model):
    """
    For fitted tree based models, get_feature_importance can be used to get the feature importance as a tidy output
    """
    X_non_text = pd.get_dummies(df[cat_cols])
    features = numeric_cols + bool_cols + list(X_non_text.columns)
    feature_importance = dict(zip(features, model.feature_importances_))
    for name, importance in sorted(feature_importance.items(), key=lambda x: x[1], reverse=True):
        print(f"{name:<30}: {importance:>6.2%}")
        print(f"\nTotal importance: {sum(feature_importance.values()):.2%}")
    return feature_importance

def model_comparison(model_performance_dict, sort_by = 'RMSE', metric = 'RMSE'):

    Rsq_list = []
    RMSE_list = []
    MAE_list = []
    for key in model_performance_dict.keys():
        Rsq_list.append(model_performance_dict[key][0])
        RMSE_list.append(model_performance_dict[key][1])
        MAE_list.append(model_performance_dict[key][2])

    props = pd.DataFrame([])

    props["R-squared"] = Rsq_list
    props["RMSE"] = RMSE_list
    props["MAE"] = MAE_list
    props.index = model_performance_dict.keys()
    props = props.sort_values(by = sort_by)

    fig, ax = plt.subplots(figsize = (12,6))

    ax.bar(props.index, props[metric], color="green")
    plt.title(metric)
    plt.xlabel('Model')
    plt.xticks(rotation = 45)
    plt.ylabel(metric)



In [0]:
X1 = whole1['selftext']
y1 = whole1['z score']

X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=1)

X_train1.values
#y_train1

# **LSTM Regression**

In [0]:
from keras.layers import BatchNormalization
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# MAX_NB_WORDS = 500
# EMBEDDING_DIM = 100
# MAX_SEQUENCE_LENGTH = 11200
# tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
# print(whole1['selftext'].values)
# print(whole1['selftext'].shape)
# print(whole1['z score'].shape)
# tokenizer.fit_on_texts(str(whole1['selftext'].values))
# word_index = tokenizer.word_index
# print('Found %s unique tokens.' % len(word_index))

# X_temp = tokenizer.texts_to_sequences(str(whole1['selftext'].values))
# X_temp = pad_sequences(X_temp, maxlen=MAX_SEQUENCE_LENGTH)
# print('Shape of data tensor:', X_temp.shape)
 

# print(X_temp.shape)
# X_temp = X_temp.transpose()
# X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(X_temp, whole1['z score'], test_size=0.2, random_state=1)

# print(X_train_lstm.shape, y_train_lstm.shape)
# print(X_test_lstm.shape,y_test_lstm.shape)
# X_test_lstm[1].shape

# #y_train1

In [0]:


# def baseline_model():
# 	# create model
# 	model = Sequential()
# 	model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train_lstm.shape[1]))
# 	model.add(LSTM(1))
# 	model.add(BatchNormalization())
# #	model.add(Dense(60, kernel_initializer='normal', activation='relu'))
# #	model.add(BatchNormalization())
# 	model.add(Dense(1, kernel_initializer='normal'))
# 	# Compile model
# 	model.compile(loss='mean_squared_error', optimizer='adam')
# 	return model
# estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=10, verbose=0)
# #batch_size = [5, 10, 20, 40, 60, 80, 100]
# #epochs = [100, 150]
# #param_grid = dict(batch_size=batch_size, epochs=epochs)
# #grid = GridSearchCV(estimator=estimator, param_grid=param_grid, n_jobs=-1, cv=3)

# estimator.fit(X_train_lstm, y_train_lstm)
# pred=estimator.predict(X_test_lstm)
# rmse = np.sqrt(mean_squared_error(y_test_lstm, pred))
# print("RMSE: %f" % (rmse))

# #grid_result = grid.fit(X_train, y_train)
# # summarize results
# #print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# # means = grid_result.cv_results_['mean_test_score']
# # stds = grid_result.cv_results_['std_test_score']
# # params = grid_result.cv_results_['params']
# # for mean, stdev, param in zip(means, stds, params):
# #     print("%f (%f) with: %r" % (mean, stdev, param))


# **Linear Regression**

In [0]:
# linear = LinearRegression()
# linear.fit(X_train,y_train)
# model_performance_dict["Linear Regression"] = model_diagnostics(linear)

# **Logistic regression**

In [0]:
# clf = LogisticRegression(random_state=0)
# clf.fit(X_train, y_train)

# # pred = clf.predict(X_test)
# # rmse = np.sqrt(mean_squared_error(y_test, pred))
# # print("RMSE: %f" % (rmse))
# model_performance_dict["Logistic Regression"] = model_diagnostics(clf)

#**Ensemble** 

In [11]:
xgb_model1 = XGBRegressor(max_depth= 9, min_child_weight=3, learning_rate = 0.05, \
                     gamma = 0.2, n_estimators=200, subsample=0.6, \
                     colsample_bytree=0.8, eta=0.1, seed=1)

xgb_model2 = XGBRegressor(max_depth= 4, min_child_weight=3, learning_rate = 0.05, \
                     gamma = 0.2, n_estimators=200, subsample=0.6, \
                     colsample_bytree=0.8, eta=0.1, seed=1)

xgb_model3 = XGBRegressor(max_depth= 6, min_child_weight=5, learning_rate = 0.05, \
                     gamma = 0.2, n_estimators=200, subsample=0.6, \
                     colsample_bytree=0.8, eta=0.1, seed=1)

xg_train, x_val, yxg_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

xgb_model1.fit(xg_train, yxg_train, early_stopping_rounds=10, eval_metric='rmse',\
               verbose=True, eval_set=[(x_val, y_val)])

xgb_model2.fit(xg_train, yxg_train, early_stopping_rounds=10, eval_metric='rmse', \
          verbose=True, eval_set=[(x_val, y_val)] )

xgb_model3.fit(xg_train, yxg_train, early_stopping_rounds=10, eval_metric='rmse', \
          verbose=True, eval_set=[(x_val, y_val)])

pred1=xgb_model1.predict(X_test)
pred2=xgb_model2.predict(X_test)
pred3=xgb_model3.predict(X_test)

finalpred=(pred1+pred2+pred3)/3
r2 = r2_score(y_test, finalpred)
rmse = np.sqrt(mean_squared_error(y_test, finalpred))
mae = mean_absolute_error(y_test, finalpred)
model_performance_dict["Ensemble Regression"] = [r2,rmse,mae]

#print("RMSE: %f" % (rmse))      
#print(f"R-Sq: {r2:.4}")
print(f"RMSE: {rmse}")
#print(f"MAE: {mae}")

[12:28:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:1.08095
Will train until validation_0-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:1.06985
[2]	validation_0-rmse:1.05891
[3]	validation_0-rmse:1.05034
[4]	validation_0-rmse:1.04178
[5]	validation_0-rmse:1.03475
[6]	validation_0-rmse:1.02784
[7]	validation_0-rmse:1.02121
[8]	validation_0-rmse:1.01573
[9]	validation_0-rmse:1.0108
[10]	validation_0-rmse:1.0061
[11]	validation_0-rmse:1.00165
[12]	validation_0-rmse:0.997238
[13]	validation_0-rmse:0.99302
[14]	validation_0-rmse:0.989644
[15]	validation_0-rmse:0.986622
[16]	validation_0-rmse:0.983567
[17]	validation_0-rmse:0.98042
[18]	validation_0-rmse:0.978177
[19]	validation_0-rmse:0.976323
[20]	validation_0-rmse:0.974579
[21]	validation_0-rmse:0.973293
[22]	validation_0-rmse:0.972005
[23]	validation_0-rmse:0.970977
[24]	validation_0-rmse:0.969756
[25]	validation_0-rmse:0.968907
[2

# Random Forest

In [14]:

rf = RandomForestRegressor(n_jobs=-1, n_estimators=70, min_samples_leaf=10, random_state = 10)
rf.fit(X_train, y_train)

# pred = rf.predict(X_test)
# rmse = np.sqrt(mean_squared_error(y_test, pred))
# print("RMSE: %f" % (rmse))
model_performance_dict["Random Forest"] = model_diagnostics(rf)

R-Sq: 0.0214
RMSE: 0.9792665109837648
MAE: 0.4738505053479515


# AdaBoost

In [15]:
ada_reg = AdaBoostRegressor( n_estimators=100,learning_rate=0.005, loss='exponential')
print(ada_reg)
#AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
#         n_estimators=100, random_state=None)
ada_reg.fit(X_train, y_train)
# pred = ada_reg.predict(X_test);
# rmse = np.sqrt(mean_squared_error(y_test, pred))
# print("RMSE: %f" % (rmse))

model_performance_dict["AdaBoost"] = model_diagnostics(ada_reg)


AdaBoostRegressor(base_estimator=None, learning_rate=0.005, loss='exponential',
                  n_estimators=100, random_state=None)
R-Sq: -0.004188
RMSE: 1.0379406921345218
MAE: 0.26439470123175934


#KNN Regressor

In [16]:
# instantiate learning model (k = 3)
knn = KNeighborsRegressor(n_neighbors=20)

# fitting the model
knn.fit(X_train, y_train)

# # predict the response
# pred = knn.predict(X_test)

# rmse = np.sqrt(mean_squared_error(y_test, pred))
# print("RMSE: %f" % (rmse))
model_performance_dict["KNN Regressor"] = model_diagnostics(knn)


R-Sq: -0.02099
RMSE: 1.0002522033077992
MAE: 0.48121905700605944


#**Neural network**

In [18]:
from keras.layers import BatchNormalization
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(60, input_dim=62, kernel_initializer='normal', activation='relu'))
	model.add(BatchNormalization())
	model.add(Dense(60, kernel_initializer='normal', activation='relu'))
	model.add(BatchNormalization())
	model.add(Dropout(0.5))
#	model.add(BatchNormalization())
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=20, verbose=0)
#batch_size = [5, 10, 20, 40, 60, 80, 100]
#epochs = [100, 150]
#param_grid = dict(batch_size=batch_size, epochs=epochs)
#grid = GridSearchCV(estimator=estimator, param_grid=param_grid, n_jobs=-1, cv=3)
#init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
#param_grid = dict(init_mode=init_mode)
#grid = GridSearchCV(estimator=estimator, param_grid=param_grid, n_jobs=-1, cv=3)
#grid_result = grid.fit(X_train, y_train)
# summarize results
#print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

estimator.fit(X_train, y_train)
model_performance_dict["Neural N Regressor"] = model_diagnostics(estimator)
#pred=estimator.predict(X_test)
#rmse = np.sqrt(mean_squared_error(y_test, pred))
#print("RMSE: %f" % (rmse))

#grid_result = grid.fit(X_train, y_train)
# summarize results
#print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))


R-Sq: -37.02
RMSE: 6.103821387339332
MAE: 0.5264592628439313


# XGBoost Regressor

In [19]:
xgb = XGBRegressor(max_depth= 6, min_child_weight=5, learning_rate = 1.05, \
                     gamma = 0.2, n_estimators=200, subsample=0.6, \
                     colsample_bytree=0.8, eta=0.1, seed=1)


xgb.fit(xg_train, yxg_train, early_stopping_rounds=20, eval_metric='rmse',\
               verbose=True, eval_set=[(x_val, y_val)])

# pred=xgb.predict(X_test)

# rmse = np.sqrt(mean_squared_error(y_test, pred))
# print("RMSE: %f" % (rmse))
model_performance_dict["XGBoost Regressor"] = model_diagnostics(xgb)

[12:44:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.980057
Will train until validation_0-rmse hasn't improved in 20 rounds.
[1]	validation_0-rmse:1.00979
[2]	validation_0-rmse:1.02069
[3]	validation_0-rmse:1.02895
[4]	validation_0-rmse:1.03338
[5]	validation_0-rmse:1.0404
[6]	validation_0-rmse:1.05216
[7]	validation_0-rmse:1.07455
[8]	validation_0-rmse:1.08346
[9]	validation_0-rmse:1.09964
[10]	validation_0-rmse:1.11923
[11]	validation_0-rmse:1.12561
[12]	validation_0-rmse:1.13323
[13]	validation_0-rmse:1.13674
[14]	validation_0-rmse:1.14594
[15]	validation_0-rmse:1.15217
[16]	validation_0-rmse:1.1576
[17]	validation_0-rmse:1.16394
[18]	validation_0-rmse:1.17062
[19]	validation_0-rmse:1.18261
[20]	validation_0-rmse:1.1824
Stopping. Best iteration:
[0]	validation_0-rmse:0.980057

R-Sq: -0.06266
RMSE: 1.0204606556642037
MAE: 0.46595037167033787


In [17]:
features = list(X_train)
#avg_impt = (xgb_model1.feature_importances_ + xgb_model2.feature_importances_+xgb_model3.feature_importances_)/3
avg_impt = ada_reg.feature_importances_ 
feature_importance = dict(zip(features, avg_impt))
for name, importance in sorted(feature_importance.items(), key=lambda x: x[1], reverse=True):
    print(f"{name:<30}: {importance:>6.2%}")
    print(f"\nTotal importance: {sum(feature_importance.values()):.2%}")


topic_10                      : 32.22%

Total importance: 100.00%
sentiment_neu_content         : 18.87%

Total importance: 100.00%
sentiment_neg_content         : 11.06%

Total importance: 100.00%
topic_32                      :  5.08%

Total importance: 100.00%
readability_content           :  4.70%

Total importance: 100.00%
topic_24                      :  3.79%

Total importance: 100.00%
topic_13                      :  2.81%

Total importance: 100.00%
topic_9                       :  1.98%

Total importance: 100.00%
topic_18                      :  1.98%

Total importance: 100.00%
topic_19                      :  1.72%

Total importance: 100.00%
topic_25                      :  1.69%

Total importance: 100.00%
topic_3                       :  1.57%

Total importance: 100.00%
topic_20                      :  1.53%

Total importance: 100.00%
topic_1                       :  1.40%

Total importance: 100.00%
topic_2                       :  1.34%

Total importance: 100.00%
sentiment_

# **Gradient Boosting Regressor**

In [11]:
gbr = GradientBoostingRegressor(n_estimators=70, max_depth=5)
gbr.fit(X_train, y_train)
model_performance_dict["Gradient Boosting Regression"] = model_diagnostics(gbr)

R-Sq: 0.1708
RMSE: 0.8244979029322437
MAE: 0.20534680818068626


# **Ploting**

In [22]:
model_performance_dict

{'AdaBoost': [0.011367674715430098, 0.9842735550724518, 0.4846373370822038],
 'Ensemble Regression': [0.022813102322410406,
  0.9785594859451454,
  0.4824262650663933],
 'Gradient Boosting Regression': [9.951380565864643e-05,
  0.9898669027867569,
  0.47005931120849775],
 'KNN Regressor': [-0.020991687176422857,
  1.0002522033077992,
  0.48121905700605944],
 'Neural N Regressor': [-37.01953544331271,
  6.103821387339332,
  0.5264592628439313],
 'Random Forest': [0.02140052555908889,
  0.9792665109837648,
  0.4738505053479515],
 'XGBoost Regressor': [-0.06266335021195513,
  1.0204606556642037,
  0.46595037167033787]}

In [0]:
model_comparison(model_performance_dict, sort_by = 'R-squared', metric = 'R-squared')
model_comparison(model_performance_dict, sort_by = 'R-squared', metric = 'MAE')
model_comparison(model_performance_dict, sort_by = 'R-squared', metric = 'RMSE')